In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [ ]:
data = pd.read_csv('/content/fb_sentiment.csv')
data = data.drop(columns='Unnamed: 0')
data.head()

,FBPost,Label
0,Drug Runners and a U.S. Senator have somethin...,O
1,"Heres a single, to add, to Kindle. Just read t...",O
2,If you tire of Non-Fiction.. Check out http://...,O
3,Ghost of Round Island is supposedly nonfiction.,O
4,Why is Barnes and Nobles version of the Kindle...,N


In [78]:
Y = data['Label']
Y

TypeError: ignored

In [ ]:
def remove_duplicates (astring) :
  newstring = astring[0]
  index = 0
  for char in astring:
    if char != newstring[index] :
      newstring += char
      index += 1
  return(newstring)

In [ ]:
posts = []
for post in data['FBPost'] :
  words = post.strip('.!?,').lower().split()
  sentence = ''
  for word in words :
    word=word.strip(',.!?:;"(){}[]<>\|/-=_£+*&^%$0123456789')
    if '@' in word :
      word = 'MENTION'
    elif 'http' in word or 'www' in word :
      word = 'LINK'
    elif len(word) > 2 :
      for i in range(len(word)-2) :
        if word[i] == word[i+1] and word[i] == word[i+2] :
          word = remove_duplicates(word)
          break
    elif '/' in word :
      word = word.split('/')
      word = word[0]+' '+word[-1]
    elif '!' in word :
      word = word.split('!')
      word = word[0]+' '+word[-1]
    elif '.' in word :
      word = word.split('.')
      word = word[0]+' '+word[-1]
    sentence += word+' '
  posts.append(sentence.strip())
posts

In [ ]:
lemm_posts = []
for post in posts :
  lemm_posts.append(''.join(Mystem().lemmatize(text=post)))
  print(post)
lemm_posts

In [73]:
nltk.download('stopwords')

tfidf = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'))
posts_tfidf = tfidf.fit_transform(lemm_posts)
print(f"Матрица на {posts_tfidf.shape[0]} документов и {posts_tfidf.shape[1]} термов")

Матрица на 1000 документов и 2660 термов


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [123]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(posts_tfidf, Y, test_size=0.2, random_state=0)

In [116]:
SVC_model = sklearn.svm.SVC(kernel='rbf')
SVC_model.fit(x_train, y_train)

SVC()

In [117]:
SVC_prediction = SVC_model.predict(x_test)

In [118]:
print(accuracy_score(SVC_prediction, y_test))
print(confusion_matrix(SVC_prediction, y_test))
print(classification_report(SVC_prediction, y_test))

0.735
[[  0   0   0]
 [  7  16   6]
 [ 14  26 131]]
              precision    recall  f1-score   support

           N       0.00      0.00      0.00         0
           O       0.38      0.55      0.45        29
           P       0.96      0.77      0.85       171

    accuracy                           0.73       200
   macro avg       0.45      0.44      0.43       200
weighted avg       0.87      0.73      0.79       200



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [124]:
mlp = MLPClassifier(hidden_layer_sizes=(5), max_iter=3000)

In [125]:
mlp.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=5, max_iter=3000)

In [126]:
MLP_prediction = mlp.predict(x_test)

In [127]:
print(accuracy_score(MLP_prediction, y_test))
print(confusion_matrix(MLP_prediction, y_test))
print(classification_report(MLP_prediction, y_test))

0.71
[[  1   0   0]
 [  3  26   9]
 [ 13  33 115]]
              precision    recall  f1-score   support

           N       0.06      1.00      0.11         1
           O       0.44      0.68      0.54        38
           P       0.93      0.71      0.81       161

    accuracy                           0.71       200
   macro avg       0.48      0.80      0.48       200
weighted avg       0.83      0.71      0.75       200

